# Analysis
***
[Alejandro Ciuba](alejandrociuba.github.io), alc307@pitt.edu
***
## Summary
This is a continuation from the previous Jupyter Notebook (`overview.ipynb`) where I begin to analyze the data in-accordance to my proposed research questions. I might change or modify the research questions here. As such, the original research question is stated at the beginning of its section and then changed appropriately throughout its section in the notebook and updated in the main `README.md` file (not in `project_plan.md`, to retain the initial ideas for this research).
***
## Python Imports & Functions

In [1]:
# Necessary imports
import nltk

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Settings
# I looked this up so everything is prettier
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Use display(dataframe) to show

%pprint

Pretty printing has been turned OFF


***
## Question 1: How Do Video Game Pragmatics Differ From Their Real Life Counterpart?
This question is extremely broad. To answer this, I realized I would need comparative statistics of real-life pragmatics data to compare against the video game corpora. At the recommendation of [Dr. Na-Rae Han](https://sites.pitt.edu/~naraehan/), I decided to gather statistics provided from the [_Corpus of Contemporary American English_ (_COCA_)](https://www.english-corpora.org/coca/), for which the [_University of Pittsburgh_](https://www.pitt.edu) has a full-license. This means, that while I am using statistics gathered from data provided by _COCA_, I will **not** be publishing any dataset from said corpus and will **only** be using statistics necessary in my research.

For this question, I have broken it up into three main hypothesis which I then test 1 by 1.

### Hypothesis: Frequent Use Of The Second Person (_you_) And Mandative Verb Conjugation
My justification for this hypothesis is that, since the game is designed around the player, the game will refer to the player the most. This means that the game (characters, text, etc.) will refer to the player character the most. While many instances might refer to the player by their "name" (e.g. _Revan_ is the name of the player's character in _KOTOR_), I believe most instances will refer to the character in the second person ("you").
According to data gotten from _COCA_ the second person pronoun (searched as '\[you\]') occurs approxiamately 17,720.87 times per million words in their _Spoken_ corpus.

Other data that might be useful in this hypothesis:
- 12,166.38 per million words (might be better than just spoken, since spoken is gotten from only TV stations -> BIAS)
- 1,473.93 per million words regarding you + modal verb ("you must," "you shall") Search: "\[you\] \_vm" This could be interesting to investigate regarding frequent commands.

### Hypothesis: Deictic words are infrequent when text/dialogue involves quests or objectives
My justification for this hypothesis is that, since games typically involve quests or objectives the player needs to complete, games, in an attempt to make things clearer for the player, often repeat themselves. They repeatedly state and name the exact locations/objectives/tasks/activities that the player must do in order to beat it.

I am creating a list of all the deictic words I could think of in English, called `deictic_words.txt` because (both annoyingly and surprisingly) `nltk` does not have a list like that already.

Firstly, to test this hypothesis, I will be using the _Torchlight II_ data as it is centered around and formatted explicitly for quest dialogue. I will then get frequency counts for the various deictic words in the dialogue.